In [1]:
import os
import pandas as pd

def CheckColumnNames(expected_column_labels,dataframe,sheetname):
    """
    First Arg: expected labels of columns as a list
    Second Arg: dataframe of the sheet
    Third Arg: The name of the sheet
    """
    for label in expected_column_labels:
        if label not in dataframe.columns:
            raise KeyError("Column '{}' could not be found in the {} sheet. Please check the column name!".format(label,sheetname))    

def ValidateSiteSheet(dataframe):
    expected_column_labels=['Name', 'area', 'slacknode', 'lat', 'long', 'ctrarea', 'primpos',
       'primneg', 'secpos', 'secneg', 'terpos', 'terneg', 'syncharea',
       'htworegion']
    CheckColumnNames(expected_column_labels,dataframe,"Site")

    exclusive_urbs_columns = ["area"]
    dataframe = dataframe.drop(exclusive_urbs_columns, axis="columns")

    dataframe = dataframe.rename(columns={"Name":"Site"})
    dataframe = dataframe.set_index(["Site"])

    columns_ordered = ['slacknode', 'lat', 'long', 'ctrarea', 'primpos', 'primneg',
       'secpos', 'secneg', 'terpos', 'terneg', 'syncharea', 'htworegion']
    dataframe = dataframe.reindex(columns=columns_ordered)
    return dataframe

#Modifies the commodity sheet and relabels the data for further use.
def ValidateCommoditySheet(dataframe):
    expected_column_labels = ['Site', 'Commodity', 'Type', 'price', 'max', 'maxperhour', 'annual',
       'losses']
    CheckColumnNames(expected_column_labels,dataframe,'Commodity')

    exclusive_urbs_columns_commodity = ["max", "maxperhour"]
    dataframe = dataframe.drop(exclusive_urbs_columns_commodity, axis='columns')

    dataframe = dataframe.rename(columns={'Commodity':'Co',"Type":"type"})
    dataframe = dataframe.set_index(["Site","Co"])
    columns_ordered=["price","annual","losses","type"]
    dataframe = dataframe.reindex(columns=columns_ordered)
    return dataframe  

#Modifies the process sheet and relabels the data for further use. 
#Incomplete
def ValidateProcessSheet(dataframe):
    #Check the column labels
    expected_column_labels = ['Site', 'Process', 'inst-cap', 'cap-lo', 'cap-up', 'max-grad',
       'min-fraction', 'inv-cost', 'fix-cost', 'var-cost', 'wacc', 'y',
       'area-per-cap', 'act-up', 'on-off', 'start-cost', 'reserve-cost', 'ru',
       'rd', 'rumax', 'rdmax', 'cotwo', 'detail', 'lambda', 'heatmax',
       'maxdeltaT', 'heatupcost', 'su', 'sd', 'hotstart', 'pdt', 'pot',
       'prepow', 'pretemp', 'preheat', 'prestate', 'precaponline', 'year']
    CheckColumnNames(expected_column_labels,dataframe,'Process')
    
    #Remove urbs exclusive columns
    exclusive_urbs_columns_process = ["cap-lo","cap-up","max-grad","inv-cost","fix-cost","var-cost","wacc","y","area-per-cap"]
    dataframe = dataframe.drop(exclusive_urbs_columns_process, axis='columns')

    #relabel columns
    dataframe = dataframe.rename(columns={"Process":"Pro", "min-fraction":"act-lo", "inst-cap": "Inst-cap"})

    #Set indexes 
    dataframe = dataframe.set_index(["Site","Pro"]) #"CoIn","CoOut" is missing
    
    return dataframe

#Modifies the transmission sheet and relabels the data for further use. 
def ValidateTransmissionSheet(dataframe):
    #Check the columns
    expected_column_labels = ['Site In', 'Site Out', 'Transmission', 'Commodity', 'eff', 'inv-cost',
       'fix-cost', 'var-cost', 'inst-cap', 'cap-lo', 'cap-up', 'wacc',
       'depreciation', 'reactance', 'difflimit', 'base_voltage',
       'cap-up-therm', 'angle-up', 'u2b', 'dc-flow', 'length', 'react-pu',
       'PSTmax', 'idx']
    CheckColumnNames(expected_column_labels,dataframe,"Transmission")

    #Drop unnecessary columns
    exclusive_urbs_columns_transmission = ["inv-cost","fix-cost","wacc","depreciation","difflimit","base_voltage"]
    dataframe = dataframe.drop(exclusive_urbs_columns_transmission, axis="columns")

    #Relabel the columns
    dataframe = dataframe.rename(columns={"Site In":"SitIn", "Site Out":"SitOut", 
    "Commodity":"Co", "cap-lo":"act-lo", "cap-up":"act-up","Transmission":"tr_type"}) 

    #Set indexes
    dataframe = dataframe.set_index(["SitIn","SitOut","Co","tr_type"])

    #Ordering
    columns_ordered = ['SitIn', 'SitOut', 'Co', 'eff', 'var-cost', 'inst-cap', 'act-lo',
       'act-up', 'reactance', 'cap-up-therm', 'angle-up', 'u2b', 'dc-flow',
       'length', 'react-pu', 'tr_type', 'PSTmax', 'idx']
    dataframe = dataframe.reindex(columns=columns_ordered)

    return dataframe

def ValidateStorageSheet(dataframe):
    #Check Columns
    expected_column_labels = ['Site', 'Storage', 'Commodity', 'inst-cap-c', 'cap-lo-c', 'cap-up-c',
       'inst-cap-p', 'cap-lo-p', 'cap-up-p', 'eff-in', 'eff-out', 'inv-cost-p',
       'inv-cost-c', 'fix-cost-p', 'fix-cost-c', 'var-cost-p', 'var-cost-c',
       'wacc', 'depreciation', 'init', 'discharge', 'ep-ratio', 'inst-cap-pi',
       'inst-cap-po', 'var-cost-pi', 'var-cost-po', 'act-lo-pi', 'act-up-pi',
       'act-lo-po', 'act-up-po', 'act-lo-c', 'act-up-c', 'precont', 'prepowin',
       'prepowout', 'ru', 'rd', 'rumax', 'rdmax', 'seasonal', 'ctr']
    CheckColumnNames(expected_column_labels,dataframe,"Storage")

    #Drop unneccesary columns
    exclusive_urbs_columns = ["cap-lo-c","cap-up-c","inst-cap-p","cap-lo-p","cap-up-p","inv-cost-p","inv-cost-c","fix-cost-p",
    "fix-cost-c","var-cost-p","wacc","discharge","ep-ratio"]
    dataframe = dataframe.drop(exclusive_urbs_columns, axis='columns')

    #Relabel columns for further use
    dataframe = dataframe.rename(columns={"Storage":"Sto","Commodity":"Co"})

    #Indexing
    dataframe = dataframe.set_index(["Site","Sto","Co"])

    #Ordering
    column_names_order = ['inst-cap-pi', 'inst-cap-po', 'inst-cap-c',
       'eff-in', 'eff-out', 'var-cost-pi', 'var-cost-po', 'var-cost-c',
       'act-lo-pi', 'act-up-pi', 'act-lo-po', 'act-up-po', 'act-lo-c',
       'act-up-c', 'precont', 'prepowin', 'prepowout', 'ru', 'rd', 'rumax',
       'rdmax', 'seasonal', 'ctr']
    dataframe = dataframe.reindex(columns=column_names_order)

    return dataframe

def ValidateDsmSheet(dataframe):
    expected_column_labels = ['Site', 'Commodity', 'delay', 'eff', 'recov', 'cap-max-do',
       'cap-max-up', 'rel-inst-cap', 'var-cost']
    CheckColumnNames(expected_column_labels,dataframe,"DSM")

    #Drop unnecesary columns
    exclusive_urbs_columns = ["cap-max-do","cap-max-up"]
    dataframe = dataframe.drop(exclusive_urbs_columns, axis="columns")

    #Relabel columns for further use
    dataframe = dataframe.rename(columns={'Commodity':"Co","recov":"recovery"})
    
    #Indexing
    dataframe = dataframe.set_index(["Site","Co"])

    #ordering
    columns_ordered=['rel-inst-cap', 'eff', 'delay', 'recovery', 'var-cost']
    dataframe = dataframe.reindex(columns=columns_ordered)
    return dataframe

def ValidateSiteNames(dfSite,dfCommodity,dfProcess,dfTransmission,dfStorage,dfDSM):
    sites = dfSite.index.tolist()
    #Check site names in commodity sheet against site sheet
    for site in dfCommodity.index.levels[0].tolist():
        if site not in sites:
            raise KeyError("The site name '{}' in Commodity sheet is not listed in the sheet 'Site'!".format(site))
    #check site names in process sheet against site sheet
    for site in dfProcess.index.levels[0].tolist():
        if site not in sites:
            raise KeyError("The site name '{}' in Process sheet is not listed in the sheet 'Site'!".format(site))
    #Check site names in transmission sheet against site sheet
    for site in dfTransmission.index.levels[0].tolist():
        if site not in sites:
            raise KeyError("The site name '{}' in Transmission sheet at the column 'Site In' is not listed in the sheet 'Site'".format(site))
    for site in dfTransmission.index.levels[1].tolist():
        if site not in sites:
            raise KeyError("The site name '{}' in Transmission sheet at the column 'Site Out' is not listed in the sheet 'Site'".format(site))
    #Check site names in Storage sheet against site sheet
    for site in dfStorage.index.levels[0].tolist():
        if site not in sites:
            raise KeyError("The site name '{}' in Storage sheet is not listed in the sheet 'Site'!".format(site))
    #Check site names in DSM sheet against site sheet
    for site in dfDSM.index.levels[0].tolist():
        if site not in sites:
            raise KeyError("The site name '{}' in DSM sheet is not listed in the sheet 'Site'!".format(site))

In [2]:
#Site
excel="data\onesyncharea.xlsx"
xls = pd.ExcelFile(excel)
sites = xls.parse('Sites', index_col=[0], convert_float=False)
print(sites.head())
print("--^--"*42)

newExcel="data\propens.xlsx"
xls = pd.ExcelFile(newExcel)
sites = xls.parse("Site", convert_float=False)
sites = ValidateSiteSheet(sites)
print(sites.head())

          slacknode        lat      long  ctrarea  primpos  primneg  secpos  \
Site                                                                          
DEU00001        1.0  47.811798  8.201898      1.0      0.0      0.0     0.0   
DEU00002        0.0  49.209994  8.645225      1.0      0.0      0.0     0.0   
DEU00003        0.0  48.972296  9.376182      1.0      0.0      0.0     0.0   
DEU00004        0.0  48.170679  9.380220      1.0      0.0      0.0     0.0   
DEU00005        0.0  48.170679  9.380220      1.0      0.0      0.0     0.0   

          secneg  terpos  terneg  syncharea  htworegion  
Site                                                     
DEU00001     0.0     0.0     0.0        1.0         0.0  
DEU00002     0.0     0.0     0.0        1.0         0.0  
DEU00003     0.0     0.0     0.0        1.0         0.0  
DEU00004     0.0     0.0     0.0        1.0         0.0  
DEU00005     0.0     0.0     0.0        1.0         1.0  
--^----^----^----^----^----^----^----^--

In [3]:
#Commodity
excel="data\onesyncharea.xlsx"
xls = pd.ExcelFile(excel)
commodities = xls.parse('Commodities', index_col=[0,1], convert_float=False)
print(commodities.head())
print('--^--'*42)

newExcel="data\propens.xlsx"
xls = pd.ExcelFile(newExcel)
commodity = xls.parse('Commodity', convert_float=False)
commodity = ValidateCommoditySheet(commodity)
print(commodity.head())

                  price        annual  losses       type
Site     Co                                             
DEU00001 Coal  21.92157  0.000000e+00     0.0  SUP-Stock
DEU00002 Coal  21.92157  0.000000e+00     0.0  SUP-Stock
DEU00003 Coal  21.92157  0.000000e+00     0.0  SUP-Stock
DEU00004 Coal  21.92157  0.000000e+00     0.0  SUP-Stock
DEU00001 Elec   0.00000  1.505621e+07     0.0        DEM
--^----^----^----^----^----^----^----^----^----^----^----^----^----^----^----^----^----^----^----^----^----^----^----^----^----^----^----^----^----^----^----^----^----^----^----^----^----^----^----^----^----^--
            price  annual  losses    type
Site Co                                  
Mid  Solar    NaN     NaN     NaN   SupIm
     Wind     NaN     NaN     NaN   SupIm
     Hydro    NaN     NaN     NaN   SupIm
     Elec     NaN     NaN     NaN  Demand
     Coal     7.0     NaN     NaN   Stock
